In [1]:
# Score short psg

In [2]:
import sys; sys.path.append('../..')
from abslithist import *

In [3]:
C=lltk.load('CanonFiction')
get_savedir(C)

'/home/ryan/github/abslithist/data/scores/v6/CanonFiction'

In [4]:
# score_corpus(lltk.load('CanonFiction'))

In [5]:
df=pd.DataFrame([y for x,y in sl.stream_results(f'{get_savedir(C)}/cache') if type(y)==dict])
df

[Slingshot] Reading results.rank=0003.jsonl: 100%|██████████| 963/963 [00:00<00:00, 40970.47it/s]


,id,path_freqs,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,Abs-Conc.MRC-Conc.C16,Abs-Conc.MRC-Imag.C16,Abs-Conc.MT-Conc.C16,Abs-Conc.Median.C16,Abs-Conc.PAV-Conc.C16,Abs-Conc.PAV-Imag.C16,...,Woman-Man.MultiWords.median,Woman-Man.SingleWords.median,Abs-Conc.LSN-Hapt.orig,Abs-Conc.LSN-Imag.orig,Abs-Conc.MRC-Conc.orig,Abs-Conc.MRC-Imag.orig,Abs-Conc.MT-Conc.orig,Abs-Conc.PAV-Conc.orig,Abs-Conc.PAV-Imag.orig,Abs-Conc.Median.orig
0,chadwyck.De Forest.Overland_A_Novel.0216,/home/ryan/lltk_data/corpora/canon_fiction/fre...,0.274232,0.177004,0.175111,0.302749,0.036010,0.157482,0.229276,0.341881,...,-0.004079,0.001009,0.273169,0.327825,0.020432,0.270232,0.307488,-0.052758,0.179137,0.210404
1,chadwyck.Dunlap.Thirty_Years_Ago_or_The_M.0229,/home/ryan/lltk_data/corpora/canon_fiction/fre...,-0.093569,-0.206798,-0.196427,-0.066813,-0.333166,-0.223982,-0.148443,-0.023008,...,0.067433,0.020493,0.141776,0.218507,-0.132717,0.144770,0.166235,-0.284553,-0.026320,0.095890
2,chadwyck.Hale.Northwood_or_Life_North_a.0269,/home/ryan/lltk_data/corpora/canon_fiction/fre...,-0.192003,-0.330672,-0.323175,-0.189691,-0.455123,-0.343786,-0.281959,-0.147422,...,0.095988,-0.002555,0.051956,0.097254,-0.224342,0.092851,0.027388,-0.426508,-0.056615,-0.003804
3,chadwyck.Gissing.The_Odd_Women_By_George_G.1207,/home/ryan/lltk_data/corpora/canon_fiction/fre...,-0.145575,-0.287529,-0.257557,-0.129240,-0.394930,-0.290859,-0.206536,-0.082936,...,0.203265,0.116102,0.075683,0.090410,-0.248393,0.045456,0.050998,-0.384832,-0.122534,0.014196
4,markmark.Cather.O_Pioneers.1387,/home/ryan/lltk_data/corpora/canon_fiction/fre...,0.548456,0.425162,0.501983,0.623070,0.310922,0.435828,0.554031,0.657819,...,0.344726,0.334898,0.567868,0.532628,0.300006,0.523444,0.603787,0.462487,0.630314,0.457532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3856,chadwyck.Fielding.Jonathan_Wild.1025,/home/ryan/lltk_data/corpora/canon_fiction/fre...,-0.214094,-0.335109,-0.340039,-0.224268,-0.455869,-0.356236,-0.289517,-0.181962,...,-0.038177,-0.094681,0.043901,0.072601,-0.321118,-0.030022,-0.044746,-0.443555,-0.157492,-0.067834
3857,chadwyck.Blackmore.Lorna_Doone_a_Romance_of.1140,/home/ryan/lltk_data/corpora/canon_fiction/fre...,0.313519,0.158367,0.221634,0.365950,0.049705,0.173264,0.269928,0.394783,...,0.111706,0.092517,0.351093,0.309955,0.029558,0.293992,0.384730,-0.043719,0.183616,0.285027
3858,chadwyck.De Forest.Overland_A_Novel.0216,/home/ryan/lltk_data/corpora/canon_fiction/fre...,0.274232,0.177004,0.175111,0.302749,0.036010,0.157482,0.229276,0.341881,...,-0.004079,0.001009,0.273169,0.327825,0.020432,0.270232,0.307488,-0.052758,0.179137,0.210404
3859,markmark.Grisham.The_Brethren.1471,/home/ryan/lltk_data/corpora/canon_fiction/fre...,0.227834,0.165527,0.196751,0.281498,0.062742,0.151020,0.252043,0.324635,...,0.053323,0.086338,0.395256,0.481224,0.224276,0.381408,0.524570,0.413701,0.508780,0.350495


In [28]:
%%write_and_run .sling.score_freqs_psg.py

import sys
sys.path.append('../..')

from abslithist import *
modernizer=get_spelling_modernizer()

dfnorms=get_allnorms()

def tknz(txt):
    p = re.compile(r"\d+|[-'a-z]+|[ ]+|\s+|[.,]+|\S+", re.I)
    slice_starts = [m.start() for m in p.finditer(txt)] + [None]
    return [txt[s:e] for s, e in zip(slice_starts, slice_starts[1:])]

def score_freqs_psg(inpd):
    old=[]
    for ld in get_psg_norms2(inpd):
        if not ld: continue
        if type(ld)!=list: continue
        vals=[d['value'] for d in ld]
        vals=pd.Series(pd.to_numeric(vals,errors='coerce'))
        tokstr=cln(detokenize([d['tok'] for d in ld]))
        old.append({
            '_window':ld[0]['_window'],
            '_i':ld[0]['_i'],
            'id':ld[0]['id'],
            'passage':tokstr.replace('\n','\\n').replace('\t','\\t'),
            'value':vals.mean(),
        })
#     return pd.DataFrame(old)
    return old
    
def cln(x):
    x=lltk.cleanstrip(lltk.clean_text(x).replace('--','||||').replace('-',' - ').replace('||||','-'))
    x=x.replace(' .','.')
    return x

def get_psg_norms2(inpd):
    path_txt=inpd['path_txt']
    window_len=inpd.get('window_len',100)
    key=inpd.get('key','Abs-Conc.Median.median')
    if not os.path.exists(path_txt): return pd.DataFrame()
    word2values=dict(zip(dfnorms.reset_index().word, dfnorms[key]))
#     print(list(word2values.items())[:10], word2values.get('the'))
    _i=0
    _w=0
    window=[]#pd.DataFrame()
    nnnan=0
    with open(path_txt,encoding='utf-8',errors='ignore') as f:
        for line in f:
            line=cln(line)
            toks=tknz(line)
            for tok in toks:
                tokl=tok.lower().strip()
                tokm=modernizer.get(tokl,tokl).lower()
                valx=word2values.get(tokl,None)
                nnnan+=len(tok)
                #if tokl and tokl[0].isalpha(): nnnan+=len(tokl)
#                 if valx is not None: nnnan+=1
                dx={
                    **{
                        '_i':_i,
                        '_window':_w,
                        'tok':tok,
                        'tok_l':tokl,
                        'tok_mod':tok,
                        'value':valx
                    },
                    **inpd
                }
#                 print(dx)
#                 stop
                window.append(dx)
                _i+=1
                if nnnan>=window_len:
                    yield window
                    window=[]
                    _w+=1
                    nnnan=0
    






In [29]:
# cln("The night - walker is cool")

In [30]:
x=pd.DataFrame(score_freqs_psg({'path_txt':C.au.Austen.SenseAndSensibilityANovel.path_txt,'window_len':1000, 'id':C.au.Austen.SenseAndSensibilityANovel.id}))
# # x.groupby('_window').mean().reset_index().plot(x='_window',y='value',kind='scatter')
# [len(y) for y in x.passage]
printm('```'+x.sort_values('value').iloc[-1].passage.replace('\\n',' ')+'```')

```. Lord! how he'll chuckle over this news! I hope he will come to - night. It will be all to one a better match for your sister. Two thousand a year without debt or drawback - except the little lovechild, indeed; aye, I had forgot her; but she may be 'prenticed out at small cost, and then what does it signify? Delaford is a nice place, I can tell you; exactly what I call a nice old fashioned place, full of comforts and conveniences; quite shut in with great garden walls that are covered with the best fruit - trees in the country: and such a mulberry tree in one corner! Lord! how Charlotte and I did stuff the only time we were there! Then, there is a dove - cote, some delightful stewponds, and a very pretty canal; and every thing, in short, that one could wish for: and, moreover, it is close to the church, and only a quarter of a mile from the turnpike - road, so 'tis never dull, for if you only go and sit up in an old yew arbour behind the house, you may see all the carriages that pass along. Oh! 'tis a nice place!```

In [24]:
printm('```'+x.sort_values('value').iloc[0].passage.replace('\\n',' ')+'```')

```for the latter . You are resolved to think him blameable, because he took leave of us with less affection than his usual behaviour has shewn . And is no allowance to be made for inadvertence, or for spirits depressed by recent disappointment? Are no probabilities to be accepted, merely because they are not certainties? Is nothing due to the man whom we have all so much reason to love, and no reason in the world to think ill of? To the possibility of motives unanswerable in themselves, though unavoidably secret for a while? And, after all, what is it you suspect him of?” “I can hardly tell you myself . - But suspicion of something unpleasant is the inevitable consequence of such an alteration as we have just witnessed in him . There is great truth, however, in what you have now urged of the allowances which ought to be made for him, and it is my wish to be candid in my judgment of every body . Willoughby may undoubtedly have very sufficient reasons for his conduct, and I will hope that he has . But it would have been more```

In [ ]:
# x.value.value_counts()

In [ ]:
# score_freqs_psg(willoughby_full)

In [ ]:
# norms=get_allnorms()
# norms

In [ ]:
C=lltk.load('CanonFiction')
objects = [
    {'id':t.id, 'path_txt':t.path_txt,'window_len':1000}
    for t in C.texts()
]
savedir=os.path.join(PATH_DATA,'psgs','short','v5',C.name)

In [ ]:
cmd=sl.shoot(
    func='score_freqs_psg',
    path_src='.sling.score_freqs_psg.py',
    objects=objects,
    savedir=savedir,
    parallel=7,
)
cmd

In [ ]:
!{cmd}

In [ ]:
if not os.path.exists(PATH_PSG_CURRENT):
    df=pd.DataFrame([d 
                     for i,ld in sl.stream_results(savedir+'/cache',progress=False)
                     for d in ld
                    if type(d)==dict])
    df=df.sort_values('value')
    save_df(df,PATH_PSG_CURRENT)
else:
    df=read_df(PATH_PSG_CURRENT)

In [ ]:
# PATH_PSG_CURRENT

In [ ]:
# showpsg(df.iloc[-10].passage)

In [ ]:
# compare_psgs

In [ ]:
# !rm $PATH_PSG_CURRENT

In [ ]:
# df

In [ ]:
# showpsg(detokenize(y.tok))

In [ ]:
# n = 50  #chunk row size
# df=bothdf
# list_df = [df.dropna()[i:i+n] for i in range(0,df.shape[0],n)]
# list_df[0]

In [ ]:
# first=bothdf[:bothdf.dropna().iloc[:51].index[-1]]
# first

In [ ]:
# print(detokenize(first.tok))

In [ ]:
# len(first.value.dropna())

In [ ]:
# 29 * 2